In [1]:
    #### TESTING AS SCRIPT
    #---------------------------------- Added: 20200422: <PS>
def entre_shock():
    mr = merge_rank('./temp/lfs_a09_pqkb_ranked_V2_entrpreneurial_20200423.csv')
    if not 'LFS_sector' in mr.columns:
        mr = mr.rename(columns={'LFS_sector_x': 'LFS_sector'})
        # get subset: a09_pqkb
    mr_subset = mr[['hhid_lfs','cc101_lno','LFS_sector','a09_pqkb','c19_pclass','demand_scale', 'w_home','E_sector']]
    mr_subset
    indexNames = mr_subset[mr_subset['a09_pqkb'] == 'nan' ].index
    # Delete these row indexes from dataFrame
    mr_subset.drop(indexNames , inplace=True)
    mr_subset = mr_subset.reset_index(drop=True)
 # get subset: c19_pclass

    indexNames2 = mr_subset[mr_subset['c19_pclass'] == 'nan' ].index

        # Delete these row indexes from dataFrame
    mr_subset.drop(indexNames2 , inplace=True)
    mr_subset = mr_subset.reset_index(drop=True)
    mr_subset

    # make new column of combined string a09 && c19:
    mr_subset['a09c19'] = mr_subset['a09_pqkb'] +'-'+mr_subset['c19_pclass']

        # enforce string:
    mr_subset['a09_pqkb'] = [str(q).strip() for q in mr_subset['a09_pqkb']] # enforce type = string
    mr_subset['LFS_sector'] = [str(q).strip() for q in mr_subset['LFS_sector']] # enforce type = string
    mr_subset['c19_pclass'] = [str(q).strip() for q in mr_subset['c19_pclass']] # enforce type = string
    mr_subset['a09c19'] = [str(q).strip() for q in mr_subset['a09c19']] # enforce type = string
    if 'E_sector' in mr_subset.columns:
        mr_subset['E_sector'] = [str(q).strip() for q in mr_subset['E_sector']] # enforce type = string

    mr_subset


    x = np.unique(mr_subset.c19_pclass)
    x

    ### create entrepreneurial table:
    #rslt_df = dataframe[dataframe['Percentage'] > 80] 
    #subsetDataFrame = dfObj[dfObj['Product'].isin(['Mangos', 'Grapes']) ]


    # or nonag_wage : private household, private establishment, govt corporation, with pay (family owned business)
    df_nonag = mr_subset[~mr_subset['c19_pclass'].isin(['Self Employed', 'Employer','Without Pay (Family owned Business)'])]
    df_nonag


    # generate fraction by  ENTREPRENEURIAL sector
    df_nonag['desc_count'] = df_nonag.groupby('a09_pqkb')['a09_pqkb'].transform('count')# count unique jobs and append to mr_subset
    df_nonag['sector_count'] = df_nonag.groupby('E_sector')['E_sector'].transform('count') #count total unique sectors and append to mr_subset
    df_nonag['sector_frac'] = df_nonag['desc_count'] / df_nonag['sector_count'] # get fraction of sector as weightin

    #### for now we will leave non-ag here

    # for entrepreneurial income: self employed, employer, withOUT pay (family owned business)
    df_ent = mr_subset[mr_subset['c19_pclass'].isin(['Self Employed', 'Employer','Without Pay (Family owned Business)'])]
    df_ent

    # generate fraction by  ENTREPRENEURIAL sector
    df_ent['desc_count'] = df_ent.groupby('a09_pqkb')['a09_pqkb'].transform('count')# count unique jobs and append to mr_subset
    df_ent['sector_count'] = df_ent.groupby('E_sector')['E_sector'].transform('count') #count total unique sectors and append to mr_subset
    df_ent['sector_frac'] = df_ent['desc_count'] / df_ent['sector_count'] # get fraction of sector as weightin





    #     # generate fraction by sector
    # mr_subset['desc_count'] = mr_subset.groupby('a09_pqkb')['a09_pqkb'].transform('count')# count unique jobs and append to mr_subset
    # mr_subset['sector_count'] = mr_subset.groupby('LFS_sector')['LFS_sector'].transform('count') #count total unique sectors and append to mr_subset
    # mr_subset['sector_frac'] = mr_subset['desc_count'] / mr_subset['sector_count'] # get fraction of sector as weighting



    #####
    # here, need to insert a new column that merges a09 and c19 -- done
    # then, drop duplicates off of this column, so that we can minimize computation

    # still need logic to build the logic for each job sector
    ## may need to restructure this whole section of code

    #####
        # drop duplicates (now that overall weighting established)
    df_ent = df_ent.drop_duplicates(subset='a09_pqkb')
    df_ent = df_ent.reset_index(drop=True)



        # generate probability and combine with relative weighting
    df_ent['partial_prob'] = np.nan
    df_ent['third_col'] = np.nan
    df_ent['dummy'] = np.nan

        # incorporate Kayenat tables into 'di' &&
        # nested logic to incorporate 0-4 scale for social distancing measures
        ## where scores of 0 & 1 result in complete job lost, due to unable to distance
    i=0
    while i < len(df_ent):

        if df_ent.demand_scale[i] == 0:

            # incorporate 0-4 scale logic:

            if df_ent.w_home[i] == 0:
                df_ent.partial_prob[i] = 0

            elif df_ent.w_home[i] == 1:
                df_ent.partial_prob[i] = 0

            else:
                df_ent.partial_prob[i] = df_ent.sector_frac[i] * (random.randint(0,50)/100)


        elif df_ent.demand_scale[i] == 0.5: 

            # incorporate 0-4 scale logic:
            if df_ent.w_home[i] == 0:
                df_ent.partial_prob[i] = 0

            elif df_ent.w_home[i] == 1:
                df_ent.partial_prob[i] = 0

            else: 
                df_ent.partial_prob[i] = df_ent.sector_frac[i] * (random.randint(50,100)/100)

        elif df_ent.demand_scale[i] == 1.0:
            df_ent.partial_prob[i] = df_ent.sector_frac[i]
        else:
            df_ent.dummy[i] = -99
        i = i + 1

    # incorporate 3rd column modifiers here:
    # if (df_ent['c19_pclass'][i] == "Gov't/Gov't Corporation"):
    #     df_ent.partial_prob[i] = 0  # essentially reverts the random uniform logic implemented above


    i = i + 1

        # remove nans in summing fields, and dummy storage
    del df_ent['dummy']

    df_ent['c19_pclass'] == "Gov't/Gov't Corporation"




    # get mean probability by sector:


    #storage['fa'] = np.nan
    storage = pd.DataFrame(columns=['fa', 'di'], index=[np.unique(df_ent.E_sector)])



    for seclist in np.unique(df_ent.E_sector): # hard-coded to existing shock table

        pp = df_ent[df_ent.E_sector == seclist]
        p4 = 1 - sum(pp.partial_prob)

        # build shock table:
        storage['fa'][seclist] = storage['fa'][seclist] + p4
        print(seclist)



    # save to separate var for testing    
    rand_weighted_shock = storage

    rand_weighted_shock
    #return(rand_weighted_shock)



    storage = pd.DataFrame(columns = ['sector','fa'])
    storage


    storage = pd.DataFrame(columns=['fa', 'di'], index=[np.unique(df_ent.E_sector)])
    storage

    pp = df_ent[df_ent.E_sector == 'Construction']
    pp

    df_ent[df_ent.E_sector == 'Wholesale and Retail'].partial_prob.sum()

    df_ent[df_ent.E_sector == 'Fishing'].partial_prob.sum()

    np.unique(df_ent.E_sector) #### the spacing is fucking up the table!

    storage = pd.DataFrame(columns=['fa', 'di'], index=[np.unique(df_ent.E_sector)])

    for seclist in np.unique(df_ent.E_sector):
        pillow = 1 - (df_ent[df_ent.E_sector == seclist].partial_prob.sum())
    #     if pillow > 1:
    #         pillow = 1
        print(pillow)
        #storage[seclist]['fa'] = df_ent[df_ent.E_sector == seclist].partial_prob.sum()
        #df.loc[0:15,'A'] = 16
        storage.loc[seclist,'fa'] = pillow


    storage

    #     tstamp = (datetime.now().strftime("%Y%m%d_%H%M"))
    #     storage.to_csv('./temp/entrep_table_problem_' +tstamp+'.csv')
    
    return(storage)

IndentationError: expected an indented block (<ipython-input-1-305347a7be5f>, line 4)